In [28]:
import tensorflow as tf

# 以下是训练模型需要的参数

num_epochs = 10
batch_size = 10

# DeepFM

本文尝试用tensorflow高阶API实现DeepFM，从而让大家明白DeepFM的原理。

DeepFM是深度CTR预估家族的一员，从名字上看似乎与FM有关，FM是浅层模型中CTR预估重要武器之一，这里DeepFM就是把Deep&Wide模型中LR部分换成了FM。

所以DeepFM模型包含两个大模块：FM和DNN，而FM又分为LR和二阶特征隐因子组合。

也就是一共三部分：

1. LR: bias + wx
2. FM: <v_i, v_j>x_i x_j
3. DNN: deep 

输入的特征我们假设都是离散型的category特征，因为实际上工作遇到的场景也是以ID等离散型特征居多。

本文一共分成三大部分：

1. 处理输入特征，假设读入的数据是libsvm格式
2. 构建DeepFM的计算图
3. 保存模型，方便tensorflow serving加载

下面进入第一部分。

# 特征处理

考虑到一般深度模型需要mini batch训练，因此，读入特征文件时，需要做打散和拆分。

加入我们的特征文件存在文本中，可能文本存在本地或者集群HDFS上都行。而且假设特征文件是稀疏的libsvm格式。这时候需要用到tensorflow的data模块来处理。


tf.data.TextLineDataset 可以读取文本行，专门用于存储稀疏特征样本的，比如文本，libsvm格式的样本。


In [42]:
train_dataset = tf.data.TextLineDataset('../../data/criteo_conversion_logs/small_train.txt')

每一行是类似这样的数据：

1 0:0:0.3651 2:1163:0.3651 3:8672:0.3651 4:2183:0.3651 5:2332:0.3651 6:185:0.3651 7:2569:0.3651 8:8131:0.3651 9:5483:0.3651 10:215:0.3651 11:1520:0.3651 12:1232:0.3651 13:2738:0.3651 14:2935:0.3651 15:5428:0.3651 17:2434:0.50000 16:7755:0.50000

其中第一个是要预测的标签。

后面的是空格分开的特征及特征值。

每一个特征由三部分构成(冒号分开)：

1. 字段ID
2. 特征ID
3. 特征值

我们要用到TensorFlow自带的一些字符串工具来解析这个文本数据。解析后得到：

1. 类别标签
2. 特征的维度
3. 特征向量（稀疏）

tf.string_split可以用来做这件事，它的参数有：

```
tf.string_split(
    source,
    delimiter=' ',
    skip_empty=True
)
```

用delimiter作为拆分字符拆分source这个字段中的字符串。返回一个SparseTensor类型的实例。

注意source字段保存的是一个形状为(1, D)的字符串Tensor实例，所以可以拆分多个字符串，如果只有一个字符串需要拆分，则要要以[string]这样的形式输入。

假如输入两个字符串：['hello world', 'a b c']，返回则是这样一个SparseTensor：

1. indices = [0, 0; 0, 1; 1, 0; 1, 1; 1, 2] 
2. shape = [2, 3]
3. values = ['hello', 'world', 'a', 'b', 'c']

也就是形如下面的稀疏张量：

["hello", "world"]

[ "a",     "b",    "c"]

是一个2 x 3的矩阵，只有(0,2)位置没有值，其他都有值，indices取值可以看出。

In [40]:
def transform_libsvm_to_tensor(line):
    # 按照空格拆分
    col = tf.string_split([line], ' ')
    # 类别标签转成数值
    label = tf.string_to_number(col.values[0], out_type=tf.float32)
    # 字段、特征、值，三者一举拆分
    ffv = tf.string_split(col.values[1:], ':')
    has_field_id = True
    if ffv.shape[1] == 2:
        has_field_id = False
    # ffv 是一个稠密矩阵，一共的行数是本样本中非零特征个数，列是3，第一列是字段ID，第二列是特征ID，第三列是取值
    # 所以可以将ffv转换成一个稠密Tensor
    ffv = tf.reshape(ffv.values, ffv.dense_shape)
    # 分离ID和特征值
    feature_ids = None
    feature_vals = None
    if has_field_id:
        _, feature_ids, feature_vals = tf.split(ffv, num_or_size_splits=3, axis=1)
    else:
        feature_ids, feature_vals = tf.split(ffv, num_or_size_splits=2, axis=1)
    feature_ids = tf.string_to_number(feature_ids, out_type=tf.int32)
    feature_vals = tf.string_to_number(feature_vals, out_type=tf.float32)
    return {"feature_ids": feature_ids, "feature_vals": feature_vals}, label

In [43]:
train_dataset = train_dataset.map(transform_libsvm_to_tensor,num_parallel_calls=10).prefetch(200) 

In [44]:
# shuffle
train_dataset = train_dataset.shuffle(buffer_size=256)

In [45]:
# 重复迭代次数那么多次
train_dataset = train_dataset.repeat(num_epochs)
# 组成每次训练的批次
train_dataset = train_dataset.batch(batch_size)
# Creates an Iterator for enumerating the elements of this dataset.
#iterator = train_dataset.make_one_shot_iterator()
#batch_features, batch_labels = iterator.get_next()

至此，第一部分完成。我们已经用tensorflow提供的接口把存储在文本中的libsvm格式数据准备成TensorFlow认识的格式。

下面进行第二部分，构建模型。

# 构建模型

构建模型又分成三步工作：

1. 网络的权重
2. 输入的特征
3. 网络计算图

我们一步一步来构建。

首先是网络权重，也是学习算法要去学习的，因此在训练过程中需要不断修改它，可变的量，要用一种特殊的张量保存：Variable。

In [49]:
class DeepFM(object):
    
    def __init__(self, params = {}):
        self._field_size = params["field_size"]
        # 特征维度
        self._feature_size = params["feature_size"]
        # FM的隐因子向量维度
        self._embedding_size = params["embedding_size"]
        # L2正则参数
        self._l2_reg = params["l2_reg"]
        # 学习率
        self._learning_rate = params["learning_rate"]
        # DNN网络层数
        self._layers  = map(int, params["deep_layers"].split(','))
        # dropout的概率
        self._dropout = map(float, params["dropout"].split(','))
        # 执行模式
        self._mode = params["mode"]
        # 批量归一化参数
        self._batch_norm_decay = params['batch_norm_decay']
    
    def create_model(self):
        # fm模型的偏置权重
        self._fm_b = tf.get_variable(name='fm_b', shape=[1],
                               initializer=tf.constant_initializer(0.0))
        # FM模型的一阶权重
        self._fm_w = tf.get_variable(name='fm_w', shape=[self._feature_size],
                               initializer=tf.glorot_normal_initializer())
        # FM模型的二阶权重
        self._fm_v = tf.get_variable(name='fm_v', shape=[self._feature_size, self._embedding_size],
                               initializer=tf.glorot_normal_initializer())
        
    
    # 前向计算
    def forward(self, features, labels):
        def batch_norm_layer(x, train_phase, scope_bn):
            bn_train = tf.contrib.layers.batch_norm(x,
                                                    decay=self._batch_norm_decay,
                                                    center=True,
                                                    scale=True,
                                                    updates_collections=None,
                                                    is_training=True,
                                                    reuse=None,
                                                    scope=scope_bn)
            bn_infer = tf.contrib.layers.batch_norm(x,
                                                    decay=self._batch_norm_decay,
                                                    center=True,
                                                    scale=True,
                                                    updates_collections=None,
                                                    is_training=False,
                                                    reuse=True,
                                                    scope=scope_bn)
            z = tf.cond(tf.cast(train_phase, tf.bool),
                        lambda: bn_train,
                        lambda: bn_infer)
            return z
        
        feature_ids  = features['feature_ids']
        feature_ids = tf.reshape(feature_ids,shape=[-1,field_size])
        feature_vals = features['feature_vals']
        feature_vals = tf.reshape(feature_vals,shape=[-1,field_size])
        
        with tf.variable_scope("fm_wb"):
            # 线性模型一阶部分计算
            feature_weights = tf.nn.embedding_lookup(self._fm_w, feature_ids)    
            y_w = tf.reduce_sum(tf.multiply(feature_weights, feature_vals),1)

        with tf.variable_scope("fm_vv"):
            # 特征的隐因子向量, 论文中的K维
            embeddings = tf.nn.embedding_lookup(self._fm_v, feature_ids)
            # 这里是按照FM论文中变形后的公式计算以提高计算效率
            feature_vals = tf.reshape(feature_vals, shape=[-1, field_size, 1])
            embeddings = tf.multiply(embeddings, feature_vals)                 
            sum_square = tf.square(tf.reduce_sum(embeddings,1))
            square_sum = tf.reduce_sum(tf.square(embeddings),1)
            y_v = 0.5 * tf.reduce_sum(tf.subtract(sum_square, square_sum),1)  

        train_phase = False
        if self._mode == tf.estimator.ModeKeys.TRAIN:
            train_phase = True

        l2_reg = tf.contrib.layers.l2_regularizer(self._l2_reg)
        with tf.variable_scope("dnn"):
            # 深度模型部分，输入为FM的隐因子向量
            deep_inputs = tf.reshape(embeddings, 
                                     shape=[-1,field_size*embedding_size])  
            for i in range(len(self._layers)):
                deep_inputs = tf.contrib.layers.fully_connected(inputs=deep_inputs,
                                                                num_outputs=layers[i], 
                                                                weights_regularizer=l2_reg,
                                                                scope='dnn%d' % i)
                if self._batch_norm:
                    deep_inputs = batch_norm_layer(deep_inputs,
                                               train_phase=train_phase,
                                               scope_bn='bn_%d' %i)
                if train_phase:
                    deep_inputs = tf.nn.dropout(deep_inputs, keep_prob=self._dropout[i])
        
            y_d = tf.contrib.layers.fully_connected(inputs=deep_inputs,
                                                   num_outputs=1,
                                                   activation_fn=tf.identity, 
                                                   weights_regularizer=l2_reg,
                                                   scope='dout')
            y_d = tf.reshape(y_d,shape=[-1])
        
        with tf.variable_scope("out"):
            y_bias = self._fm_b * tf.ones_like(y_d, dtype=tf.float32)      # None * 1
            y = y_bias + y_w + y_v + y_d
            prediction = tf.sigmoid(y)

        predictions={"prediction": prediction}
         # 批量预测模式则导出预测结果
        if self._mode == tf.estimator.ModeKeys.PREDICT:
            export_outputs = {tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                              tf.estimator.export.PredictOutput(predictions)}
            return tf.estimator.EstimatorSpec(mode=self._mode,
                                              predictions=predictions,
                                              export_outputs=export_outputs)       
        
        # 无论是训练模式还是评价模式，都需要计算损失
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y, labels=labels))  \
                                + l2_reg * tf.nn.l2_loss(self._fm_w)  \
                                + l2_reg * tf.nn.l2_loss(self._fm_v)

        # 模型评估模式则计算AUC
        if self._mode == tf.estimator.ModeKeys.EVAL:
            eval_metric_ops = {
                "auc": tf.metrics.auc(labels, prediction)
            }

            return tf.estimator.EstimatorSpec(mode=self._mode,
                                              predictions=predictions,
                                              loss=loss,
                                              eval_metric_ops=eval_metric_ops)        
        # 训练模型则计算损失
        if self._mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.AdamOptimizer(learning_rate=self._learning_rate,
                                               beta1=0.832,
                                               beta2=0.987,
                                               epsilon=1e-8)
            train_operator = optimizer.minimize(loss, global_step=tf.train.get_global_step())

            return tf.estimator.EstimatorSpec(mode=self._mode,
                                              predictions=predictions,
                                              loss=loss,
                                              train_op=train_operator)